# Cliodynamics Data Exploration
The basis data for this class contains numerous socio-economic indicators tracked over time of the US economy. The goal of this notebook is to play around with the data, find correlations and discover useful predictors of economic prosperity, or of macroscopic socio-economic phase transitions.

In [1]:
import numpy as np
from numpy import random as rnd
from matplotlib import pyplot as plt
import warnings,datetime,os,calendar,time,sys,math

import torch
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd

import dash
import dash_html_components as html
import dash_core_components as dcc

<ipython-input-1-ecb7a009e068>:13: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-1-ecb7a009e068>:14: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


## Data Container Class
This class loads the data and conatins some useful functions to visualize and understand the data.

In [2]:
import pandas as pd
from matplotlib import pyplot as plt

pd.options.mode.chained_assignment = None
pd.options.plotting.backend = 'plotly'
class Economy:
    # Initalize data and metadata
    def __init__(self,path_to_businessdata='../../../TF_Data/Dropbox/TF_data/BusinessMacroHistory/data2/'):
        self.bf = pd.read_csv(path_to_businessdata+'data.csv')
        self.md = pd.read_csv(path_to_businessdata+'metadata.csv')
        return
    # Plot one time series
    def plot_timeseries_by_index(self,ix):
        # Select timeseries with index
        df = self.bf[self.bf['time_series_code']==self.md.iloc[ix].time_series_code]
        # Extract timeseries description
        desc = '({0}): {1} - {2}\n[{3}]'.format(self.md.iloc[ix].time_series_code,self.md.iloc[ix].cat_desc,self.md.iloc[ix].dt_desc,self.md.iloc[ix].dt_unit.strip())
        # Convert to float and date sequence
        df['value'] = df['value'].astype('float')
        df['date'] = pd.to_datetime(df['date'])
        # Plot
        df[['date','value']].set_index('date').plot()
        plt.title(desc)
        plt.show()
        return
    # Extract one single time series
    def extract_single_timeseries(self,ix):
        df = self.bf[self.bf['time_series_code']==self.md.iloc[ix].time_series_code].drop(columns=['time_series_code'])
        df['date'] = pd.to_datetime(df['date'])
        df = df.set_index('date')
        df['value'] = df['value'].astype(float)
        df = df.rename(columns={'value':self.md.iloc[ix].time_series_code}) 
        return df

In [3]:
E = Economy()

C:\Users\z003mxpm\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [ ]:
# Create app
app = dash.Dash(__name__)

# Define layout
app.layout = html.Div(
    children=[
        html.H1(children='Welcome User - Cliodynamical Evaluations Data'),
        html.Div(children='''
            Visualization of several characteristic time series.
        '''),
        dcc.Graph(
            id='Time Series',
            figure=E.extract_single_timeseries(4).plot()
        )
    ]
)

# Run app
if __name__ == '__main__': app.run_server(debug=True,use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [55]:
df = E.bf

df = df[df.time_series_code.isin(df.time_series_code.unique()[rnd.randint(0,100,15)])].pivot(
    index=['time_series_code'],
    columns=['date'],
    values=['value']
)
df.columns = df.columns.to_series().str.join('_')

In [56]:
df.transpose().astype(float).corr()

time_series_code,ESTIMATE_OFFMAR_WE,ESTIMATE_OTH_NE,ESTIMATE_OWNOCC_US,ESTIMATE_RENT_US,ESTIMATE_RENT_WE,ESTIMATE_RNTOCC_MW,ESTIMATE_TOTAL_WE,ESTIMATE_YRVAC_WE,RATE_E_RVR_US,RATE_HOR_MW,RATE_RVR_SO,RATE_SAHOR_US_adj
time_series_code,,,,,,,,,,,,
ESTIMATE_OFFMAR_WE,1.000000,0.873202,0.596905,0.373524,0.371721,0.799666,0.877320,0.872254,0.729203,-0.787517,-0.229428,-0.653857
ESTIMATE_OTH_NE,0.873202,1.000000,0.538275,0.181959,0.156749,0.886517,0.890062,0.669101,0.534865,-0.881468,-0.400876,-0.780565
ESTIMATE_OWNOCC_US,0.596905,0.538275,1.000000,0.615584,0.417750,0.486304,0.787164,0.694073,0.818300,-0.403364,0.091806,-0.184553
ESTIMATE_RENT_US,0.373524,0.181959,0.615584,1.000000,0.896830,-0.059484,0.311722,0.749090,0.630570,0.097858,0.742308,0.312519
ESTIMATE_RENT_WE,0.371721,0.156749,0.417750,0.896830,1.000000,-0.041979,0.243590,0.726121,0.471481,0.057560,0.652992,0.216565
ESTIMATE_RNTOCC_MW,0.799666,0.886517,0.486304,-0.059484,-0.041979,1.000000,0.904850,0.499880,0.419218,-0.991736,-0.653044,-0.937457
ESTIMATE_TOTAL_WE,0.877320,0.890062,0.787164,0.311722,0.243590,0.904850,1.000000,0.735824,0.691033,-0.866232,-0.352444,-0.728494
ESTIMATE_YRVAC_WE,0.872254,0.669101,0.694073,0.749090,0.726121,0.499880,0.735824,1.000000,0.818437,-0.476420,0.231198,-0.273347
RATE_E_RVR_US,0.729203,0.534865,0.818300,0.630570,0.471481,0.419218,0.691033,0.818437,1.000000,0.262267,0.652389,0.363676
